In [1]:
# Import the dependencies.
import pandas as pd
import gmaps
import gmaps.datasets
import requests
import matplotlib


class InvalidPointException(Exception):
    pass

# Import the API key.
from config import g_key

In [2]:
# Store the CSV you saved created in part one into a DataFrame.
city_data_df = pd.read_csv("weather_data/WeatherPy_vacation.csv")
city_data_df.head()

,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
0,3,Hermanus,ZA,66.00,-34.42,19.23,clear sky,Misty Waves Boutique Hotel
1,4,Altamira,BR,82.40,-3.20,-52.21,broken clouds,Requinte Hotel
2,5,Mataura,NZ,51.01,-46.19,168.86,overcast clouds,Ellie's Villa
3,6,Kapaa,US,66.20,22.08,-159.32,clear sky,Sheraton Kauai Resort at Coconut Beach
4,7,Bereda,ES,54.00,43.27,-7.54,few clouds,Pazo de Verdes


In [3]:
# Get the data types
city_data_df.dtypes

City_ID                  int64
City                    object
Country                 object
Max Temp               float64
Lat                    float64
Lng                    float64
Current Description     object
Hotel Name              object
dtype: object

In [4]:
#Sort Data Fram for cities near each other, sort by lat

city_data_df.sort_values(by=["Lat","Country"])


,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
18,27,Ushuaia,AR,50.00,-54.80,-68.30,light intensity drizzle,Albatros Hotel
79,106,Punta Arenas,CL,55.40,-53.15,-70.92,broken clouds,Hotel Dreams Del Estrecho
16,23,Bluff,NZ,51.01,-46.60,168.33,scattered clouds,Bluff Homestead - Guesthouse & Campervan Park
173,224,Kaitangata,NZ,55.99,-46.28,169.85,overcast clouds,Kaitangata Motor Camp
2,5,Mataura,NZ,51.01,-46.19,168.86,overcast clouds,Ellie's Villa
...,...,...,...,...,...,...,...,...
241,313,Talnakh,RU,14.00,69.49,88.40,shower snow,Talnakh Hotel
204,261,Vardo,NO,26.60,70.37,31.11,broken clouds,Vardø Hotel AS
292,380,Havoysund,NO,33.80,71.00,24.66,fog,Arctic View Glamping & Restaurant
322,423,Mehamn,NO,33.80,71.04,27.85,broken clouds,Mehamn Arctic Hotel AS


In [5]:
#filter Data Frame and create separate data fram for each target city

# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Country"] == "US") & \
                                       (city_data_df["Lat"] >= 42.10) & \
                                       (city_data_df["Lat"] <= 42.49)]

# Create the output file (CSV).
output_data_file = "weather_data/Itinerary_vacation.csv"
# Export the City_Data into a CSV.
preferred_cities_df.to_csv(output_data_file, index_label="Random Order")

preferred_cities_df.head(10)

,City_ID,City,Country,Max Temp,Lat,Lng,Current Description,Hotel Name
20,29,Jamestown,US,48.20,42.10,-79.24,clear sky,Hampton Inn & Suites Jamestown
106,139,Palmer,US,54.00,42.16,-72.33,few clouds,Wedgewood Motel
294,382,Altamont,US,43.00,42.21,-121.74,overcast clouds,Microtel Inn & Suites by Wyndham Klamath Falls
422,548,Madison Heights,US,51.01,42.49,-83.11,clear sky,Red Roof Inn Detroit - Royal Oak/Madison Heights


In [6]:
# Set the parameters to search for a hotel in Paris to be editted later
params = {
    "location": "48.8566, 2.3522"}
   
# Iterate through the DataFrame.
for index, row in preferred_cities_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

In [7]:
# Configure gmaps to use your Google API key.
gmaps.configure(api_key=g_key)

In [8]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""
# Store the DataFrame Row.
preferred_cities_info = [info_box_template.format(**row) for index, row in preferred_cities_df.iterrows()]

print(preferred_cities_info)

['\n<dl>\n<dt>Hotel Name</dt><dd>Hampton Inn & Suites Jamestown</dd>\n<dt>City</dt><dd>Jamestown</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>clear sky</dd>\n<dt>Max Temp</dt><dd>48.2 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Wedgewood Motel</dd>\n<dt>City</dt><dd>Palmer</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>few clouds</dd>\n<dt>Max Temp</dt><dd>54.0 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Microtel Inn & Suites by Wyndham Klamath Falls</dd>\n<dt>City</dt><dd>Altamont</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>overcast clouds</dd>\n<dt>Max Temp</dt><dd>43.0 °F</dd>\n</dl>\n', '\n<dl>\n<dt>Hotel Name</dt><dd>Red Roof Inn Detroit - Royal Oak/Madison Heights</dd>\n<dt>City</dt><dd>Madison Heights</dd>\n<dt>Country</dt><dd>US</dd>\n<dt>Current Description</dt><dd>clear sky</dd>\n<dt>Max Temp</dt><dd>51.01 °F</dd>\n</dl>\n']


In [15]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)


fig = gmaps.figure(center=(51.5, 0.1), zoom_level=7)


locations = preferred_cities_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=preferred_cities_info)

fig.add_layer(marker_layer)



fig

Figure(layout=FigureLayout(height='420px'))

In [11]:
import gmaps
import gmaps.datasets
gmaps.configure(api_key=g_key)


fig = gmaps.figure(center=(51.5, 0.1), zoom_level=7)


#Latitude-longitude pairs
Jamestown = (42.1, -79.24)
Palmer = (42.16, -72.33)
Madison_Heights = (42.49, -83.11)
Altamont = (42.21, -121.74)

fig = gmaps.figure()
Palmer2Altamont = gmaps.directions_layer(Palmer, Altamont, waypoints=[Jamestown, Madison_Heights])
fig.add_layer(Palmer2Altamont)


fig

Figure(layout=FigureLayout(height='420px'))

In [14]:
locations = preferred_cities_df[["Lat", "Lng"]]

marker_layer = gmaps.marker_layer(locations, info_box_content=preferred_cities_info)

fig.add_layer(marker_layer)

fig

Figure(layout=FigureLayout(height='420px'))